In [1]:
import numpy as np
import torch
import datasets
import pandas as pd
# import editdistance
import transformers
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from accelerate import Accelerator
from sklearn.model_selection import StratifiedShuffleSplit
import os
from tqdm import tqdm
import re




===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/user/ksharma/miniconda3/envs/ks_tars_thesis did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existe

In [2]:
# results_gpt3_P1_df.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/GPT3_SS/results_gpt3_P1_df.csv")
# results_gpt3_P2_df.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/GPT3_SS/results_gpt3_P2_df.csv")
# results_gpt3_P3_df.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/GPT3_SS/results_gpt3_P3_df.csv")

results_gpt3_P3_df = pd.read_csv('/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/GPT3_SS/results_gpt3_P3_df.csv')

In [3]:
results_gpt3_P3_df['Problem type'].value_counts()

Problem type
OWA     98
MWA     66
MC      50
HOWA    30
MDC     30
HMC      8
HMDC     5
NONE     3
MOWA     1
Name: count, dtype: int64

In [5]:
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_row', None)

import pandas as pd
df = pd.read_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/GPT3_SS/GPT3_analyse.csv")
df = df[df['USAGE FLAG'] == 1]
values_to_match = [557, 629, 701, 738, 793, 812, 820,830]

values_to_match_WRONG = [585, 619, 625, 678, 542, 728, 722, 749, 754, 761, 813]

new_df = df[df['idx'].isin(values_to_match)]
new_df

# # Part 1: "idx" column values from 186 to 285
# results_gpt3_P1_df = df[(df['idx'] >= 186) & (df['idx'] <= 285)]

# # Part 2: "idx" column values from 286 to 535
# results_gpt3_P2_df = df[(df['idx'] >= 286) & (df['idx'] <= 535)]

# Part 3: "idx" column values from 536 to 835
results_gpt3_P3_df = df[(df['idx'] >= 536) & (df['idx'] <= 835)]

In [13]:
values_to_match_WRONG = [585, 619, 625, 678, 542, 728, 722, 749, 754, 761, 813]

new_df_WRONG = df[df['idx'].isin(values_to_match_WRONG)]
new_df_WRONG

,idx,input,generated_text,expected_text,wrong GPT3,Problem type,USAGE FLAG
356,542,"Als <SoftSkill_C>kommunikative</SoftSkill_C>, <SoftSkill_C>kontaktfreudige</SoftSkill_C> und <SoftSkill>verhandlungsgeschickte Persönlichkeit</SoftSkill> sind auch mündliche Fremdsprachenkenntnisse von Vorteil. ->","Als <SoftSkill>kommunikative Persönlichkeit</SoftSkill>, <SoftSkill>kontaktfreudige Persönlichkeit</SoftSkill> und <SoftSkill>verhandlungsgeschickte Persönlichkeit</SoftSkill> sind auch mündliche Fremdsprachenkenntnisse von Vorteil.","Als <SoftSkill>kommunikative Persönlichkeit</SoftSkill>, <SoftSkill>kontaktfreudige Persönlichkeit</SoftSkill> und <SoftSkill>verhandlungsgeschickte Persönlichkeit</SoftSkill> sind auch mündliche Fremdsprachenkenntnisse von Vorteil.",1,MC,1.0
399,585,"Sie können <SoftSkill>Veränderungen erfolgreich</SoftSkill> und <SoftSkill>motivierend initiieren</SoftSkill>, <SoftSkill_C>umsetzen</SoftSkill_C> und <SoftSkill_C>begleiten</SoftSkill_C>. ->","Sie können <SoftSkill>Veränderungen erfolgreich initiieren</SoftSkill> und <SoftSkill>motivierend initiieren</SoftSkill>, <SoftSkill>Veränderungen umsetzen</SoftSkill> und <SoftSkill>motivierend begleiten</SoftSkill>.","Sie können <SoftSkill>Veränderungen erfolgreich initiieren</SoftSkill> und <SoftSkill>motivierend initiieren</SoftSkill>, <SoftSkill>Veränderungen umsetzen</SoftSkill> und <SoftSkill>Veränderungen begleiten</SoftSkill>.",0,MWA/MC,1.0
433,619,Sie besitzen ein\n<SoftSkill>gutes Verständnis für Geodaten</SoftSkill> und <SoftSkill_C>zeitgemässe Technologien</SoftSkill_C>. ->,Sie besitzen ein <SoftSkill>gutes Verständnis für Geodaten</SoftSkill> und <SoftSkill>ein zeitgemässe Technologien</SoftSkill>.,Sie besitzen ein <SoftSkill>gutes Verständnis für Geodaten</SoftSkill> und <SoftSkill>gutes Verständnis für zeitgemässe Technologien</SoftSkill>.,0,MWA,1.0
439,625,<SoftSkill>Hohe Flexibilität bezüglich unregelmässiger Arbeitszeiten</SoftSkill> und bei <SoftSkill_C>kurzfristigen Arbeitseinsätzen</SoftSkill_C> ->,<SoftSkill>Hohe Flexibilität bezüglich unregelmässiger Arbeitszeiten</SoftSkill> und <SoftSkill>hohe Flexibilität bezüglich kurzfristiger Arbeitseinsätze</SoftSkill>,<SoftSkill>Hohe Flexibilität bezüglich unregelmässiger Arbeitszeiten</SoftSkill> und <SoftSkill>hohe Flexibilität bei kurzfristigen Arbeitseinsätzen</SoftSkill>,0,MWA,1.0
492,678,Die pädagogische Arbeit planen Sie <SoftSkill_C>zielorientiert</SoftSkill_C> und <SoftSkill>einfühlsam in Partner-schaft mit den Eltern und dem Team</SoftSkill>. ->,Die pädagogische Arbeit planen Sie <SoftSkill>zielorientiert in Partner-schaft mit den Eltern und dem Team</SoftSkill> und <SoftSkill>einfühlsam</SoftSkill>.,Die pädagogische Arbeit planen Sie <SoftSkill>zielorientiert in Partner-schaft mit den Eltern und dem Team</SoftSkill> und <SoftSkill>einfühlsam in Partner-schaft mit den Eltern und dem Team</SoftSkill>.,0,MWA,1.0
535,722,"Sie sind eine <SoftSkill_C>aufgeschlossen</SoftSkill_C>, <SoftSkill_C>engagiert</SoftSkill_C>\nund <SoftSkill>zuverlässig arbeitende Person</SoftSkill> mit einer <SoftSkill_C>stark service</SoftSkill_C> ->","Sie sind eine <SoftSkill>aufgeschlossen arbeitende Person</SoftSkill>, <SoftSkill>engagiert arbeitende Person</SoftSkill> und <SoftSkill>zuverlässig arbeitende Person</SoftSkill> mit einer <SoftSkill>stark service arbeitende Person</SoftSkill>.","Sie sind eine <SoftSkill>aufgeschlossen arbeitende Person</SoftSkill>, <SoftSkill>engagiert arbeitende Person</SoftSkill> und <SoftSkill>zuverlässig arbeitende Person</SoftSkill> mit einer <SoftSkill>stark service</SoftSkill>.",0,MDC,1.0
541,728,• <SoftSkill>Unternehmerisches Verständnis</SoftSkill> und <SoftSkill_C>Denken</SoftSkill_C> ->,<SoftSkill>Unternehmerisches Verständnis</SoftSkill> und <SoftSkill>denken</SoftSkill>.,• <SoftSkill>Unternehmerisches Verständnis</SoftSkill> und <SoftSkill>Unternehmerisches denken</SoftSkill>.,0,OWA,1.0
562,749,"Du <SoftSkill>arbeitest exakt</SoftSkill>, <SoftSkill_C>verantwortungsbewusst</S

In [14]:
new_df_WRONG = new_df_WRONG.iloc[:, [1, 2, 3]]
latex_table = new_df_WRONG.to_latex(index=False)
print(latex_table)

\begin{tabular}{lll}
\toprule
input & generated_text & expected_text \\
\midrule
 Als <SoftSkill_C>kommunikative</SoftSkill_C>, <SoftSkill_C>kontaktfreudige</SoftSkill_C> und <SoftSkill>verhandlungsgeschickte Persönlichkeit</SoftSkill> sind auch mündliche Fremdsprachenkenntnisse von Vorteil. -> &  Als <SoftSkill>kommunikative Persönlichkeit</SoftSkill>, <SoftSkill>kontaktfreudige Persönlichkeit</SoftSkill> und <SoftSkill>verhandlungsgeschickte Persönlichkeit</SoftSkill> sind auch mündliche Fremdsprachenkenntnisse von Vorteil. &  Als <SoftSkill>kommunikative Persönlichkeit</SoftSkill>, <SoftSkill>kontaktfreudige Persönlichkeit</SoftSkill> und <SoftSkill>verhandlungsgeschickte Persönlichkeit</SoftSkill> sind auch mündliche Fremdsprachenkenntnisse von Vorteil. \\
 Sie können <SoftSkill>Veränderungen erfolgreich</SoftSkill> und <SoftSkill>motivierend initiieren</SoftSkill>, <SoftSkill_C>umsetzen</SoftSkill_C> und <SoftSkill_C>begleiten</SoftSkill_C>. -> &  Sie können <SoftSkill>Veränderung

In [12]:
n_df = new_df.iloc[:, [1, 2]]
latex_table = n_df.to_latex(index=False)
print(latex_table)
# n_df

\begin{tabular}{ll}
\toprule
input & generated_text \\
\midrule
 Sie <SoftSkill>arbeiten gerne fachbezogen</SoftSkill> und verstehen es, <SoftSkill>Jugendliche im selbständigen Arbeiten zu coachen</SoftSkill> und zu <SoftSkill_C>motivieren</SoftSkill_C>. -> &  Sie <SoftSkill>arbeiten gerne fachbezogen</SoftSkill> und verstehen es, <SoftSkill>Jugendliche im selbständigen Arbeiten zu coachen</SoftSkill> und <SoftSkill>Jugendliche im selbständigen Arbeiten zu motivieren</SoftSkill>. \\
 Durch Ihr <SoftSkill_C>engagiertes</SoftSkill_C>, <SoftSkill_C>freundliches</SoftSkill_C> und <SoftSkill>kundenorientiertes Auftreten</SoftSkill> <SoftSkill>tragen Sie als belastbare</SoftSkill> und <SoftSkill>flexible Persönlichkeit</SoftSkill> einen wesentlichen Teil zur erfolgreichen Teamzusammenarbeit bei. -> &  Durch Ihr <SoftSkill>engagiertes Auftreten</SoftSkill>, <SoftSkill>freundliches Auftreten</SoftSkill> und <SoftSkill>kundenorientiertes Auftreten</SoftSkill> <SoftSkill>tragen Sie als belastbar

In [6]:
# results_gpt3_P3_df = results_gpt3_P3_df.drop(columns='USAGE FLAG', axis = 0)
# results_gpt3_P3_df = results_gpt3_P3_df.drop(columns='wrong GPT3', axis = 0)
results_gpt3_P3_df['expected_skills_all'] = results_gpt3_P3_df['expected_text'].str.findall(r'<SoftSkill>(.*?)<\/SoftSkill')
results_gpt3_P3_df['generated_skills_all'] = results_gpt3_P3_df['generated_text'].str.findall(r'(?:SoftSkill>|SoftSkill_C>)(.*?)(?:</SoftSkill>|</SoftSkill_C>)')
results_gpt3_P3_df.head(4)

/tmp/ipykernel_1360799/3370328973.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_gpt3_P3_df['expected_skills_all'] = results_gpt3_P3_df['expected_text'].str.findall(r'<SoftSkill>(.*?)<\/SoftSkill')
/tmp/ipykernel_1360799/3370328973.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_gpt3_P3_df['generated_skills_all'] = results_gpt3_P3_df['generated_text'].str.findall(r'(?:SoftSkill>|SoftSkill_C>)(.*?)(?:</SoftSkill>|</SoftSkill_C>)')


,idx,input,generated_text,expected_text,wrong GPT3,Problem type,USAGE FLAG,expected_skills_all,generated_skills_all
350,536,• <SoftSkill_C>autonomes</SoftSkill_C> und <SoftSkill>initiatives Arbeiten</SoftSkill> ->,<SoftSkill>autonomes Arbeiten</SoftSkill> und <SoftSkill>initiatives Arbeiten</SoftSkill>,<SoftSkill>autonomes Arbeiten</SoftSkill> und <SoftSkill>initiatives Arbeiten</SoftSkill>,1,OWA,1.0,"[autonomes Arbeiten, initiatives Arbeiten]","[autonomes Arbeiten, initiatives Arbeiten]"
351,537,"Darüber hinaus verfügen Sie über sehr gute Englischkenntnisse und die <SoftSkill>Fähigkeit, ein Team zu begeistern</SoftSkill> und zu <SoftSkill_C>motivieren</SoftSkill_C>. ->","Darüber hinaus verfügen Sie über sehr gute Englischkenntnisse und die <SoftSkill>Fähigkeit, ein Team zu begeistern</SoftSkill> und <SoftSkill>die Fähigkeit, ein Team zu motivieren</SoftSkill>.","Darüber hinaus verfügen Sie über sehr gute Englischkenntnisse und die <SoftSkill>Fähigkeit, ein Team zu begeistern</SoftSkill> und <SoftSkill>die Fähigkeit, ein Team zu motivieren</SoftSkill>.",1,MWA,1.0,"[Fähigkeit, ein Team zu begeistern, die Fähigkeit, ein Team zu motivieren]","[Fähigkeit, ein Team zu begeistern, die Fähigkeit, ein Team zu motivieren]"
352,538,Ein <SoftSkill>ausgedehntes Flair für Zahlen</SoftSkill> und <SoftSkill_C>vernetztes Denken</SoftSkill_C> runden Ihr Profil ab. ->,Ein <SoftSkill>ausgedehntes Flair für Zahlen</SoftSkill> und <SoftSkill>vernetztes Denken</SoftSkill> runden Ihr Profil ab.,Ein <SoftSkill>ausgedehntes Flair für Zahlen</SoftSkill> und <SoftSkill>ausgedehntes Flair für vernetztes Denken</SoftSkill> runden Ihr Profil ab.,0,NONE,1.0,"[ausgedehntes Flair für Zahlen, ausgedehntes Flair für vernetztes Denken]","[ausgedehntes Flair für Zahlen, vernetztes Denken]"
353,539,• <SoftSkill_C>selbständiges</SoftSkill_C> und <SoftSkill>eigenverantwortliches Arbeiten</SoftSkill> ->,<SoftSkill>Selbständiges Arbeiten</SoftSkill> und <SoftSkill>eigenverantwortliches Arbeiten</SoftSkill>,<SoftSkill>Selbständiges Arbeiten</SoftSkill> und <SoftSkill>eigenverantwortliches Arbeiten</SoftSkill>,1,OWA,1.0,"[Selbständiges Arbeiten, eigenverantwortliches Arbeiten]","[Selbständiges Arbeiten, eigenverantwortliches Arbeiten]"


In [250]:
# results_gpt3_P3_df[results_gpt3_P3_df['wrong GPT3'] == 0]

## GET TEST DATA 

In [7]:

import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge


# TESTING SAVED MODELS

In [8]:
import pandas as pd
import re
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_row', None)

# results_gpt3_P3_df = pd.read_csv("/home/user/ksharma/ks_thesis/ma-thesis/bloom_peft_lora/bloom_7bn_results_ORIG_15epochs.csv")
# results_gpt3_P3_df = results_gpt3_P3_df[results_gpt3_P3_df['valid'] == 1]
# results_gpt3_P3_df = results_gpt3_P3_df.drop(columns='Unnamed: 0', axis = 0)
# results_gpt3_P3_df = results_gpt3_P3_df.drop(columns='expected_skills', axis = 0)
# results_gpt3_P3_df = results_gpt3_P3_df.drop(columns='generated_skills', axis = 0)

# Function to extract skills between "<SoftSkill_C>" and "</SoftSkill_C>"
def extract_skills_c(text):
    return re.findall(r'<SoftSkill_C>(.*?)<\/SoftSkill_C', text)

# Function to extract skills between "<SoftSkill>"
def extract_skills(text):
    return re.findall(r'<SoftSkill>(.*?)<\/SoftSkill', text)

# Apply the functions to 'input', 'expected_text', and 'generated_text' columns
results_gpt3_P3_df['input_skills_C'] = results_gpt3_P3_df['input'].apply(extract_skills_c)
# df['input_skills'] = df['input'].apply(extract_skills)
results_gpt3_P3_df['input_skills_all'] = results_gpt3_P3_df['input'].str.findall(r'(?:<SoftSkill>|<SoftSkill_C>)(.*?)(?:</SoftSkill|</SoftSkill_C)')
# results_gpt3_P3_df['expected_skills_all'] = results_gpt3_P3_df['expected_text'].str.findall(r'<SoftSkill>(.*?)<\/SoftSkill')
# results_gpt3_P3_df['generated_skills_all'] = results_gpt3_P3_df['generated_text'].str.findall(r'SoftSkill>(.*?)/SoftSkill>')
# results_gpt3_P3_df


/tmp/ipykernel_1360799/3352822248.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_gpt3_P3_df['input_skills_C'] = results_gpt3_P3_df['input'].apply(extract_skills_c)
/tmp/ipykernel_1360799/3352822248.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_gpt3_P3_df['input_skills_all'] = results_gpt3_P3_df['input'].str.findall(r'(?:<SoftSkill>|<SoftSkill_C>)(.*?)(?:</SoftSkill|</SoftSkill_C)')


In [253]:
# results_gpt3_P3_df[results_gpt3_P3_df['orig_idx'] == 1910]

In [9]:
# Function to find the corresponding skills in 'expected_text' and 'generated_text'
def find_C_expected_skills(row):
    corresponding_skills = []
    for skill_c in row['input_skills_C']:
        if skill_c in row['input_skills_all']:
            idx = row['input_skills_all'].index(skill_c)
            corresponding_skills.append(row['expected_skills_all'][idx])
            # corresponding_skills.append(row['generated_skills_new'][idx])
    return corresponding_skills

# def find_C_generated_skills(row):
#     corresponding_skills = []
#     for skill_c, generated_skill in zip(row['input_skills_C'], row['generated_skills_all']):
#         if skill_c in row['input_skills_all']:
#             idx = row['input_skills_all'].index(skill_c)
#             corresponding_skills.append(generated_skill)
#     return corresponding_skills

def find_C_generated_skills(row):
    corresponding_skills = []
    # print(f"input_skills_C = {row['input_skills_C']}")
    # print(f"end=input_skills_all = {row['input_skills_all']}")
    # print(f"generated_skills_all = {row['generated_skills_all']}")
    if len(row['input_skills_all']) == len(row['generated_skills_all']):
        for skill_c in row['input_skills_C']:
            if skill_c in row['input_skills_all']:
                idx = row['input_skills_all'].index(skill_c)
            corresponding_skills.append(row['generated_skills_all'][idx])
    return corresponding_skills

# Apply the function to create 'corresponding_skills' column
results_gpt3_P3_df['expected_skills_C'] = results_gpt3_P3_df.apply(find_C_expected_skills, axis=1)
results_gpt3_P3_df['generated_skills_C'] = results_gpt3_P3_df.apply(find_C_generated_skills, axis=1)


/tmp/ipykernel_1360799/3761818267.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_gpt3_P3_df['expected_skills_C'] = results_gpt3_P3_df.apply(find_C_expected_skills, axis=1)
/tmp/ipykernel_1360799/3761818267.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_gpt3_P3_df['generated_skills_C'] = results_gpt3_P3_df.apply(find_C_generated_skills, axis=1)


In [255]:
# corresponding_skills = []

# input_C = ['Fähigkeit', 'motivieren']
# input_all = ['Fähigkeit', 'ein Team zu begeistern', 'motivieren']
# indexes = []
# generated_all = ['Fähigkeit in ama', 'ein Team zu begeistern', 'die Fähigkeit, ein Team zu motivieren']
# for skill_c in input_C:
#     print(skill_c)
#     if skill_c in input_all:
#         idx = input_all.index(skill_c)
#     corresponding_skills.append(generated_all[idx])
# corresponding_skills

In [10]:
results_gpt3_P3_df['num_skills_all'] = results_gpt3_P3_df.apply(lambda row: len(row['expected_skills_all']) if isinstance(row['expected_skills_all'], list) else 0, axis=1)
results_gpt3_P3_df['num_skills_C'] = results_gpt3_P3_df.apply(lambda row: len(row['expected_skills_C']) if isinstance(row['expected_skills_C'], list) else 0, axis=1)

results_gpt3_P3_df['num_gen_skills_all'] = results_gpt3_P3_df.apply(lambda row: len(row['generated_skills_all']) if isinstance(row['generated_skills_all'], list) else 0, axis=1)
results_gpt3_P3_df['num_gen_skills_C'] = results_gpt3_P3_df.apply(lambda row: len(row['generated_skills_C']) if isinstance(row['generated_skills_C'], list) else 0, axis=1)


results_gpt3_P3_df.head(3)

/tmp/ipykernel_1360799/3549903362.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_gpt3_P3_df['num_skills_all'] = results_gpt3_P3_df.apply(lambda row: len(row['expected_skills_all']) if isinstance(row['expected_skills_all'], list) else 0, axis=1)
/tmp/ipykernel_1360799/3549903362.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_gpt3_P3_df['num_skills_C'] = results_gpt3_P3_df.apply(lambda row: len(row['expected_skills_C']) if isinstance(row['expected_skills_C'], list) else 0, axis=1)
/t

,idx,input,generated_text,expected_text,wrong GPT3,Problem type,USAGE FLAG,expected_skills_all,generated_skills_all,input_skills_C,input_skills_all,expected_skills_C,generated_skills_C,num_skills_all,num_skills_C,num_gen_skills_all,num_gen_skills_C
350,536,• <SoftSkill_C>autonomes</SoftSkill_C> und <SoftSkill>initiatives Arbeiten</SoftSkill> ->,<SoftSkill>autonomes Arbeiten</SoftSkill> und <SoftSkill>initiatives Arbeiten</SoftSkill>,<SoftSkill>autonomes Arbeiten</SoftSkill> und <SoftSkill>initiatives Arbeiten</SoftSkill>,1,OWA,1.0,"[autonomes Arbeiten, initiatives Arbeiten]","[autonomes Arbeiten, initiatives Arbeiten]",[autonomes],"[autonomes, initiatives Arbeiten]",[autonomes Arbeiten],[autonomes Arbeiten],2,1,2,1
351,537,"Darüber hinaus verfügen Sie über sehr gute Englischkenntnisse und die <SoftSkill>Fähigkeit, ein Team zu begeistern</SoftSkill> und zu <SoftSkill_C>motivieren</SoftSkill_C>. ->","Darüber hinaus verfügen Sie über sehr gute Englischkenntnisse und die <SoftSkill>Fähigkeit, ein Team zu begeistern</SoftSkill> und <SoftSkill>die Fähigkeit, ein Team zu motivieren</SoftSkill>.","Darüber hinaus verfügen Sie über sehr gute Englischkenntnisse und die <SoftSkill>Fähigkeit, ein Team zu begeistern</SoftSkill> und <SoftSkill>die Fähigkeit, ein Team zu motivieren</SoftSkill>.",1,MWA,1.0,"[Fähigkeit, ein Team zu begeistern, die Fähigkeit, ein Team zu motivieren]","[Fähigkeit, ein Team zu begeistern, die Fähigkeit, ein Team zu motivieren]",[motivieren],"[Fähigkeit, ein Team zu begeistern, motivieren]","[die Fähigkeit, ein Team zu motivieren]","[die Fähigkeit, ein Team zu motivieren]",2,1,2,1
352,538,Ein <SoftSkill>ausgedehntes Flair für Zahlen</SoftSkill> und <SoftSkill_C>vernetztes Denken</SoftSkill_C> runden Ihr Profil ab. ->,Ein <SoftSkill>ausgedehntes Flair für Zahlen</SoftSkill> und <SoftSkill>vernetztes Denken</SoftSkill> runden Ihr Profil ab.,Ein <SoftSkill>ausgedehntes Flair für Zahlen</SoftSkill> und <SoftSkill>ausgedehntes Flair für vernetztes Denken</SoftSkill> runden Ihr Profil ab.,0,NONE,1.0,"[ausgedehntes Flair für Zahlen, ausgedehntes Flair für vernetztes Denken]","[ausgedehntes Flair für Zahlen, vernetztes Denken]",[vernetztes Denken],"[ausgedehntes Flair für Zahlen, vernetztes Denken]",[ausgedehntes Flair für vernetztes Denken],[vernetztes Denken],2,1,2,1


In [257]:
# results_gpt3_P3_df.loc[results_gpt3_P3_df['num_skills_C'] == 0, 'input'] = results_gpt3_P3_df.loc[results_gpt3_P3_df['num_skills_C'] == 0, 'input'].apply(lambda x: x + '>')

In [258]:
results_gpt3_P3_df[results_gpt3_P3_df['num_skills_C']==0]

,Unnamed: 0,idx,input,generated_text,expected_text,wrong GPT3,Problem type,expected_skills_all,generated_skills_all,input_skills_C,input_skills_all,expected_skills_C,generated_skills_C,num_skills_all,num_skills_C,num_gen_skills_all,num_gen_skills_C,percent_skills_match_all,percent_skills_match_C,num_skills_match_all,num_skills_match_C,cleaned_generated_text,cleaned_expected_text,sentence_levenshtein,rouge_scores_skill_C,bleu_scores_skill_C,rouge1_avg_skill_C,rouge2_avg_skill_C,rougeL_avg_skill_C,bleu_avg_skill_C,levenshtein_skill_C,rouge_scores_skill_all,bleu_scores_skill_all,rouge1_avg_skill_all,rouge2_avg_skill_all,rougeL_avg_skill_all,bleu_avg_skill_all,levenshtein_skill_all,mismatched_ALL


In [11]:

from collections import Counter

# Create a new column 'skills_match_score'
results_gpt3_P3_df['percent_skills_match_all'] = 0.0
results_gpt3_P3_df['percent_skills_match_C'] = 0.0

results_gpt3_P3_df['num_skills_match_all'] = 0.0
results_gpt3_P3_df['num_skills_match_C'] = 0.0

# Iterate over the rows of the DataFrame
for index, row in results_gpt3_P3_df.iterrows():
    expected_skills_all = set(row['expected_skills_all'])
    generated_skills_all = set(row['generated_skills_all'])
    
    expected_skills_C = set(row['expected_skills_C'])
    generated_skills_C = set(row['generated_skills_C'])

    count_gen_C = Counter(generated_skills_C)
    count_exp_C = Counter(expected_skills_C)

    # Find the common elements between list1 and list2
    common_elements_C = set(expected_skills_C).intersection(set(generated_skills_C))

    # Find the number of common elements, considering duplicates
    num_common_elements_C = sum(min(count_gen_C[element], count_exp_C[element]) for element in common_elements_C)

    count_gen_all = Counter(generated_skills_all)
    count_exp_all = Counter(expected_skills_all)

    # Find the common elements between list1 and list2
    common_elements_all = set(expected_skills_all).intersection(set(generated_skills_all))

    # Find the number of common elements, considering duplicates
    num_common_elements_all = sum(min(count_gen_all[element], count_exp_all[element]) for element in common_elements_all)



    # Calculate the percentage of matching skills
    if len(expected_skills_all) > 0:
        match_score_all = len(expected_skills_all.intersection(generated_skills_all)) / len(expected_skills_all) * 100
    else:
        match_score_all = 0.0
    # Calculate the percentage of matching skills
    if len(expected_skills_C) > 0:
        match_score_C = len(expected_skills_C.intersection(generated_skills_C)) / len(expected_skills_C) * 100
    else:
        match_score_C = 0.0
    
    # Assign the match score to the 'skills_match_score' column
    results_gpt3_P3_df.at[index, 'percent_skills_match_all'] = match_score_all
    results_gpt3_P3_df.at[index, 'percent_skills_match_C'] = match_score_C
    results_gpt3_P3_df.at[index, 'num_skills_match_all'] = num_common_elements_all
    results_gpt3_P3_df.at[index, 'num_skills_match_C'] = num_common_elements_C

/tmp/ipykernel_1360799/2450052421.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_gpt3_P3_df['percent_skills_match_all'] = 0.0
/tmp/ipykernel_1360799/2450052421.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_gpt3_P3_df['percent_skills_match_C'] = 0.0
/tmp/ipykernel_1360799/2450052421.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

In [12]:
results_gpt3_P3_df[results_gpt3_P3_df['percent_skills_match_all'] == 100]['Problem type'].value_counts()

Problem type
OWA         94
MWA         59
MC          47
HOWA        29
MDC         15
HMC          7
MC/MWA       6
HMDC         3
NONE         2
OWA/NONE     1
MOWA         1
HMC/HMDC     1
HMDC/HMC     1
Name: count, dtype: int64

In [261]:
results_gpt3_P3_df[results_gpt3_P3_df['percent_skills_match_all'] == 100]['Problem type'].value_counts()

Problem type
OWA     94
MWA     59
MC      47
HOWA    29
MDC     22
HMC      7
HMDC     5
NONE     2
MOWA     1
Name: count, dtype: int64

In [15]:
results_gpt3_P3_df[results_gpt3_P3_df['Problem type'] == 'NONE']


,idx,input,generated_text,expected_text,wrong GPT3,Problem type,USAGE FLAG,expected_skills_all,generated_skills_all,input_skills_C,...,expected_skills_C,generated_skills_C,num_skills_all,num_skills_C,num_gen_skills_all,num_gen_skills_C,percent_skills_match_all,percent_skills_match_C,num_skills_match_all,num_skills_match_C
352,538,Ein <SoftSkill>ausgedehntes Flair für Zahlen</SoftSkill> und <SoftSkill_C>vernetztes Denken</SoftSkill_C> runden Ihr Profil ab. ->,Ein <SoftSkill>ausgedehntes Flair für Zahlen</SoftSkill> und <SoftSkill>vernetztes Denken</SoftSkill> runden Ihr Profil ab.,Ein <SoftSkill>ausgedehntes Flair für Zahlen</SoftSkill> und <SoftSkill>ausgedehntes Flair für vernetztes Denken</SoftSkill> runden Ihr Profil ab.,0,NONE,1.0,"[ausgedehntes Flair für Zahlen, ausgedehntes Flair für vernetztes Denken]","[ausgedehntes Flair für Zahlen, vernetztes Denken]",[vernetztes Denken],...,[ausgedehntes Flair für vernetztes Denken],[vernetztes Denken],2,1,2,1,50.0,0.0,1.0,0.0
421,607,"Gleichzeitig wollen Sie sie fördern und <SoftSkill_C>fordern</SoftSkill_C> und haben <SoftSkill>Spass daran, mit ihnen zahlreiche Aktivitäten zu unternehmen</SoftSkill>. ->","Gleichzeitig wollen Sie sie fördern und <SoftSkill>fordern</SoftSkill> und haben <SoftSkill>Spass daran, mit ihnen zahlreiche Aktivitäten zu unternehmen</SoftSkill>.","Gleichzeitig wollen Sie sie fördern und <SoftSkill>fordern</SoftSkill> und haben <SoftSkill>Spass daran, mit ihnen zahlreiche Aktivitäten zu unternehmen</SoftSkill>.",1,NONE,1.0,"[fordern, Spass daran, mit ihnen zahlreiche Aktivitäten zu unternehmen]","[fordern, Spass daran, mit ihnen zahlreiche Aktivitäten zu unternehmen]",[fordern],...,[fordern],[fordern],2,1,2,1,100.0,100.0,2.0,1.0
592,779,"Ihr Profil:\n<SoftSkill>Lieben Sie es, selbständig zu arbeiten</SoftSkill> und Ihren <SoftSkill_C>Arbeitsbereich optimal zu organisieren</SoftSkill_C>? ->","Ihr Profil: <SoftSkill>Lieben Sie es, selbständig zu arbeiten</SoftSkill> und <SoftSkill>lieben Sie es, Ihren Arbeitsbereich optimal zu organisieren</SoftSkill>?","Ihr Profil: <SoftSkill>Lieben Sie es, selbständig zu arbeiten</SoftSkill> und <SoftSkill>lieben Sie es, Ihren Arbeitsbereich optimal zu organisieren</SoftSkill>?",1,NONE,1.0,"[Lieben Sie es, selbständig zu arbeiten, lieben Sie es, Ihren Arbeitsbereich optimal zu organisieren]","[Lieben Sie es, selbständig zu arbeiten, lieben Sie es, Ihren Arbeitsbereich optimal zu organisieren]",[Arbeitsbereich optimal zu organisieren],...,"[lieben Sie es, Ihren Arbeitsbereich optimal zu organisieren]","[lieben Sie es, Ihren Arbeitsbereich optimal zu organisieren]",2,1,2,1,100.0,100.0,2.0,1.0


In [262]:
import Levenshtein
import re 
def remove_sp_char(sent):
    modified_text = re.sub(r'(<)?(?:\/?(?:SoftSkill(?:_C)?))?(>)', '', sent)
    modified_text = re.sub(r'\n', ' ', modified_text)
    modified_text = re.sub(r'\r', ' ', modified_text)
    return modified_text

results_gpt3_P3_df['cleaned_generated_text'] = results_gpt3_P3_df['generated_text'].apply(remove_sp_char)
results_gpt3_P3_df['cleaned_expected_text'] = results_gpt3_P3_df['expected_text'].apply(remove_sp_char)
results_gpt3_P3_df['sentence_levenshtein'] = results_gpt3_P3_df.apply(lambda row: Levenshtein.distance(row['cleaned_generated_text'], row['cleaned_expected_text']), axis=1)
# results_gpt3_P3_df['skills_levenshtein'] = results_gpt3_P3_df.apply(lambda row: Levenshtein.distance(str(row['generated_skills']), str(row['expected_skills'])), axis=1)

In [263]:
# Define functions to calculate Rouge and BLEU scores
def calculate_rouge_scores(expected_list, generated_list):
    rouge = Rouge()
    rouge_scores = []
    min_length = min(len(expected_list), len(generated_list))
    for i in range(min_length):
        expected = expected_list[i]
        generated = generated_list[i]
        scores = rouge.get_scores(expected, generated)
        rouge_scores.append(scores)
    return rouge_scores

def calculate_bleu_score(expected_list, generated_list):
    smoothing = SmoothingFunction().method0
    bleu_scores = []
    for expected, generated in zip(expected_list, generated_list):
        if len(expected_list) == len(generated_list):
            bleu_score = sentence_bleu([expected], generated, smoothing_function=smoothing)
            bleu_scores.append(bleu_score)
    return bleu_scores

def calculate_levenshtein_distance(expected_list, generated_list):
    levenshtein_distances = []
    for expected, generated in zip(expected_list, generated_list):
        if len(expected_list) == len(generated_list):
            distance = Levenshtein.distance(expected, generated)
            levenshtein_distances.append(distance)
    return levenshtein_distances

# Calculate the Rouge and BLEU scores for each row and store them in new columns
results_gpt3_P3_df['rouge_scores_skill_C'] = results_gpt3_P3_df.apply(lambda row: calculate_rouge_scores(row['expected_skills_C'], row['generated_skills_C']), axis=1)
results_gpt3_P3_df['bleu_scores_skill_C'] = results_gpt3_P3_df.apply(lambda row: calculate_bleu_score(row['expected_skills_C'], row['generated_skills_C']), axis=1)

# Calculate the average scores for each pair individually
results_gpt3_P3_df['rouge1_avg_skill_C'] = results_gpt3_P3_df['rouge_scores_skill_C'].apply(lambda x: sum(score[0]['rouge-1']['f'] for score in x) / len(x) if len(x) > 0 else-1)
results_gpt3_P3_df['rouge2_avg_skill_C'] = results_gpt3_P3_df['rouge_scores_skill_C'].apply(lambda x: sum(score[0]['rouge-2']['f'] for score in x) / len(x) if len(x) > 0 else -1)
results_gpt3_P3_df['rougeL_avg_skill_C'] = results_gpt3_P3_df['rouge_scores_skill_C'].apply(lambda x: sum(score[0]['rouge-l']['f'] for score in x) / len(x) if len(x) > 0 else -1)
results_gpt3_P3_df['bleu_avg_skill_C'] = results_gpt3_P3_df['bleu_scores_skill_C'].apply(lambda x: sum(x) / len(x) if len(x) > 0 else -1)

# Calculate the Levenshtein distances for each pair individually and store the sum in a new column
results_gpt3_P3_df['levenshtein_skill_C'] = results_gpt3_P3_df.apply(lambda row: sum(calculate_levenshtein_distance(row['expected_skills_C'], row['generated_skills_C'])), axis=1)


# Calculate the Rouge and BLEU scores for each row and store them in new columns
results_gpt3_P3_df['rouge_scores_skill_all'] = results_gpt3_P3_df.apply(lambda row: calculate_rouge_scores(row['expected_skills_all'], row['generated_skills_all']), axis=1)
results_gpt3_P3_df['bleu_scores_skill_all'] = results_gpt3_P3_df.apply(lambda row: calculate_bleu_score(row['expected_skills_all'], row['generated_skills_all']), axis=1)

# Calculate the average scores for each pair individually
results_gpt3_P3_df['rouge1_avg_skill_all'] = results_gpt3_P3_df['rouge_scores_skill_all'].apply(lambda x: sum(score[0]['rouge-1']['f'] for score in x) / len(x) if len(x) > 0 else -1)
results_gpt3_P3_df['rouge2_avg_skill_all'] = results_gpt3_P3_df['rouge_scores_skill_all'].apply(lambda x: sum(score[0]['rouge-2']['f'] for score in x) / len(x) if len(x) > 0 else -1)
results_gpt3_P3_df['rougeL_avg_skill_all'] = results_gpt3_P3_df['rouge_scores_skill_all'].apply(lambda x: sum(score[0]['rouge-l']['f'] for score in x) / len(x) if len(x) > 0 else -1)
results_gpt3_P3_df['bleu_avg_skill_all'] = results_gpt3_P3_df['bleu_scores_skill_all'].apply(lambda x: sum(x) / len(x) if len(x) > 0 else -1)

# Calculate the Levenshtein distances for each pair individually and store the sum in a new column
results_gpt3_P3_df['levenshtein_skill_all'] = results_gpt3_P3_df.apply(lambda row: sum(calculate_levenshtein_distance(row['expected_skills_all'], row['generated_skills_all'])), axis=1)

# results_gpt3_P3_df[results_gpt3_P3_df['orig_idx']==694]



In [264]:
results_gpt3_P3_df.columns

Index(['Unnamed: 0', 'idx', 'input', 'generated_text', 'expected_text',
       'wrong GPT3', 'Problem type', 'expected_skills_all',
       'generated_skills_all', 'input_skills_C', 'input_skills_all',
       'expected_skills_C', 'generated_skills_C', 'num_skills_all',
       'num_skills_C', 'num_gen_skills_all', 'num_gen_skills_C',
       'percent_skills_match_all', 'percent_skills_match_C',
       'num_skills_match_all', 'num_skills_match_C', 'cleaned_generated_text',
       'cleaned_expected_text', 'sentence_levenshtein', 'rouge_scores_skill_C',
       'bleu_scores_skill_C', 'rouge1_avg_skill_C', 'rouge2_avg_skill_C',
       'rougeL_avg_skill_C', 'bleu_avg_skill_C', 'levenshtein_skill_C',
       'rouge_scores_skill_all', 'bleu_scores_skill_all',
       'rouge1_avg_skill_all', 'rouge2_avg_skill_all', 'rougeL_avg_skill_all',
       'bleu_avg_skill_all', 'levenshtein_skill_all', 'mismatched_ALL'],
      dtype='object')

In [265]:
# results_gpt3_P3_df.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/bloom-7b1-lora-ORIG-stopseq20230717114608.csv")


In [266]:
results_gpt3_P3_df['Problem type'].value_counts()

Problem type
OWA     98
MWA     66
MC      50
HOWA    30
MDC     30
HMC      8
HMDC     5
NONE     3
MOWA     1
Name: count, dtype: int64

In [267]:
pd.set_option('display.max_columns', None)  # Display all columns
# results_gpt3_P3_df[results_gpt3_P3_df['num_skills_C'] != results_gpt3_P3_df['num_gen_skills_C']]
results_gpt3_P3_df[results_gpt3_P3_df['num_skills_all'] != results_gpt3_P3_df['num_gen_skills_all']]
results_gpt3_P3_df['mismatched_ALL'] = results_gpt3_P3_df['num_skills_all'] != results_gpt3_P3_df['num_gen_skills_all']
results_gpt3_P3_df[results_gpt3_P3_df['num_skills_all'] != results_gpt3_P3_df['num_gen_skills_all']].head(3)

,Unnamed: 0,idx,input,generated_text,expected_text,wrong GPT3,Problem type,expected_skills_all,generated_skills_all,input_skills_C,input_skills_all,expected_skills_C,generated_skills_C,num_skills_all,num_skills_C,num_gen_skills_all,num_gen_skills_C,percent_skills_match_all,percent_skills_match_C,num_skills_match_all,num_skills_match_C,cleaned_generated_text,cleaned_expected_text,sentence_levenshtein,rouge_scores_skill_C,bleu_scores_skill_C,rouge1_avg_skill_C,rouge2_avg_skill_C,rougeL_avg_skill_C,bleu_avg_skill_C,levenshtein_skill_C,rouge_scores_skill_all,bleu_scores_skill_all,rouge1_avg_skill_all,rouge2_avg_skill_all,rougeL_avg_skill_all,bleu_avg_skill_all,levenshtein_skill_all,mismatched_ALL
263,620,807,"Ein <SoftSkill_C>freundliches</SoftSkill_C> <SoftSkill_C>verkaufs-</SoftSkill_C> und <SoftSkill>kundenorientiertes Verhalten</SoftSkill> mit <SoftSkill>guten Umgangsformen</SoftSkill> sind Ihnen selbstverständlich, wobei Sie auch <SoftSkill>in schwierigen Situationen aufgrund Ihrer strukturierten Arbeitsweise den Überblick behalten</SoftSkill>. ->","Ein <SoftSkill>freundliches Verkaufsorientiertes Verhalten</SoftSkill> und <SoftSkill>kundenorientiertes Verhalten</SoftSkill> mit <SoftSkill>guten Umgangsformen</SoftSkill> sind Ihnen selbstverständlich, wobei Sie auch <SoftSkill>in schwierigen Situationen aufgrund Ihrer strukturierten Arbeitsweise den Überblick behalten</SoftSkill>.","Ein <SoftSkill>freundliches Verhalten</SoftSkill> und <SoftSkill> Verkaufsorientiertes Verhalten</SoftSkill> und <SoftSkill>kundenorientiertes Verhalten</SoftSkill> mit <SoftSkill>guten Umgangsformen</SoftSkill> sind Ihnen selbstverständlich, wobei Sie auch <SoftSkill>in schwierigen Situationen aufgrund Ihrer strukturierten Arbeitsweise den Überblick behalten</SoftSkill>.",0,HMC,"[freundliches Verhalten, Verkaufsorientiertes Verhalten, kundenorientiertes Verhalten, guten Umgangsformen, in schwierigen Situationen aufgrund Ihrer strukturierten Arbeitsweise den Überblick behalten]","[freundliches Verkaufsorientiertes Verhalten, kundenorientiertes Verhalten, guten Umgangsformen, in schwierigen Situationen aufgrund Ihrer strukturierten Arbeitsweise den Überblick behalten]","[freundliches, verkaufs-]","[freundliches, verkaufs-, kundenorientiertes Verhalten, guten Umgangsformen, in schwierigen Situationen aufgrund Ihrer strukturierten Arbeitsweise den Überblick behalten]","[freundliches Verhalten, Verkaufsorientiertes Verhalten]",[],5,2,4,0,60.0,0.0,3.0,0.0,"Ein freundliches Verkaufsorientiertes Verhalten und kundenorientiertes Verhalten mit guten Umgangsformen sind Ihnen selbstverständlich, wobei Sie auch in schwierigen Situationen aufgrund Ihrer strukturierten Arbeitsweise den Überblick behalten.","Ein freundliches Verhalten und Verkaufsorientiertes Verhalten und kundenorientiertes Verhalten mit guten Umgangsformen sind Ihnen selbstverständlich, wobei Sie auch in schwierigen Situationen aufgrund Ihrer strukturierten Arbeitsweise den Überblick behalten.",15,[],[],-1.0,-1.0,-1.0,-1.0,0,"[[{'rouge-1': {'r': 0.6666666666666666, 'p': 1.0, 'f': 0.7999999952000001}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.6666666666666666, 'p': 1.0, 'f': 0.7999999952000001}}], [{'rouge-1': {'r': 0.5, 'p': 0.5, 'f': 0.4999999950000001}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.5, 'p': 0.5, 'f': 0.4999999950000001}}], [{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}], [{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}]]",[],0.325000,0.000000,0.325000,-1.0,0,True
269,626,813,"Zudem sind Sie es <SoftSkill_C>gewohnt</SoftSkill_C>, <SoftSkill_C>selbstständig</SoftSkill_C> und <SoftSkill>verantwortungsbewusst zu arbeiten</SoftSkill>. ->","Zudem sind Sie es <SoftSkill>gewohnt, selbstständig und verantwortungsbewusst zu arbeiten</SoftSkill>.","Zudem sind Sie es <SoftSkill>gewohnt zu a

In [268]:
results_C = results_gpt3_P3_df[['idx', 'input', 'expected_text', 'generated_text', 'input_skills_C', 'expected_skills_C', 'generated_skills_C',
                'num_skills_C', 'num_gen_skills_C', 'percent_skills_match_C', 'num_skills_match_C',
                'rouge_scores_skill_C', 'bleu_scores_skill_C', 'rouge1_avg_skill_C',
                'rouge2_avg_skill_C', 'rougeL_avg_skill_C', 'bleu_avg_skill_C', 'levenshtein_skill_C', 'mismatched_ALL','Problem type']]

# Create "results_ALL" DataFrame
results_ALL = results_gpt3_P3_df[['idx', 'input', 'expected_text', 'generated_text', 'input_skills_all', 'expected_skills_all', 'generated_skills_all',
                  'num_skills_all', 'num_gen_skills_all', 'percent_skills_match_all', 'num_skills_match_all',
                  'rouge_scores_skill_all', 'bleu_scores_skill_all', 'rouge1_avg_skill_all',
                  'rouge2_avg_skill_all', 'rougeL_avg_skill_all', 'bleu_avg_skill_all', 'levenshtein_skill_all', 'mismatched_ALL','Problem type']]

# Create "results_SENTENCE" DataFrame
results_SENTENCE = results_gpt3_P3_df[['idx', 'input', 'expected_text', 'generated_text', 'cleaned_generated_text', 'cleaned_expected_text', 'sentence_levenshtein','Problem type']]

# # Export the three DataFrames to separate CSV files (optional)
results_C.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/GPT3_SS/gpt_part_3_results_C.csv", index=False)
results_ALL.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/GPT3_SS/gpt_part_3_results_ALL.csv", index=False)
results_SENTENCE.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/GPT3_SS/gpt_part_3_results_SENTENCE.csv", index=False)

In [269]:
results_ALL[results_ALL['mismatched_ALL']==True].head(5)

,idx,input,expected_text,generated_text,input_skills_all,expected_skills_all,generated_skills_all,num_skills_all,num_gen_skills_all,percent_skills_match_all,num_skills_match_all,rouge_scores_skill_all,bleu_scores_skill_all,rouge1_avg_skill_all,rouge2_avg_skill_all,rougeL_avg_skill_all,bleu_avg_skill_all,levenshtein_skill_all,mismatched_ALL,Problem type
263,807,"Ein <SoftSkill_C>freundliches</SoftSkill_C> <SoftSkill_C>verkaufs-</SoftSkill_C> und <SoftSkill>kundenorientiertes Verhalten</SoftSkill> mit <SoftSkill>guten Umgangsformen</SoftSkill> sind Ihnen selbstverständlich, wobei Sie auch <SoftSkill>in schwierigen Situationen aufgrund Ihrer strukturierten Arbeitsweise den Überblick behalten</SoftSkill>. ->","Ein <SoftSkill>freundliches Verhalten</SoftSkill> und <SoftSkill> Verkaufsorientiertes Verhalten</SoftSkill> und <SoftSkill>kundenorientiertes Verhalten</SoftSkill> mit <SoftSkill>guten Umgangsformen</SoftSkill> sind Ihnen selbstverständlich, wobei Sie auch <SoftSkill>in schwierigen Situationen aufgrund Ihrer strukturierten Arbeitsweise den Überblick behalten</SoftSkill>.","Ein <SoftSkill>freundliches Verkaufsorientiertes Verhalten</SoftSkill> und <SoftSkill>kundenorientiertes Verhalten</SoftSkill> mit <SoftSkill>guten Umgangsformen</SoftSkill> sind Ihnen selbstverständlich, wobei Sie auch <SoftSkill>in schwierigen Situationen aufgrund Ihrer strukturierten Arbeitsweise den Überblick behalten</SoftSkill>.","[freundliches, verkaufs-, kundenorientiertes Verhalten, guten Umgangsformen, in schwierigen Situationen aufgrund Ihrer strukturierten Arbeitsweise den Überblick behalten]","[freundliches Verhalten, Verkaufsorientiertes Verhalten, kundenorientiertes Verhalten, guten Umgangsformen, in schwierigen Situationen aufgrund Ihrer strukturierten Arbeitsweise den Überblick behalten]","[freundliches Verkaufsorientiertes Verhalten, kundenorientiertes Verhalten, guten Umgangsformen, in schwierigen Situationen aufgrund Ihrer strukturierten Arbeitsweise den Überblick behalten]",5,4,60.0,3.0,"[[{'rouge-1': {'r': 0.6666666666666666, 'p': 1.0, 'f': 0.7999999952000001}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.6666666666666666, 'p': 1.0, 'f': 0.7999999952000001}}], [{'rouge-1': {'r': 0.5, 'p': 0.5, 'f': 0.4999999950000001}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.5, 'p': 0.5, 'f': 0.4999999950000001}}], [{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}], [{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}]]",[],0.325000,0.000000,0.325000,-1.0,0,True,HMC
269,813,"Zudem sind Sie es <SoftSkill_C>gewohnt</SoftSkill_C>, <SoftSkill_C>selbstständig</SoftSkill_C> und <SoftSkill>verantwortungsbewusst zu arbeiten</SoftSkill>. ->","Zudem sind Sie es <SoftSkill>gewohnt zu arbeiten</SoftSkill>, <SoftSkill>selbstständig zu arbeiten</SoftSkill> und <SoftSkill>verantwortungsbewusst zu arbeiten</SoftSkill>.","Zudem sind Sie es <SoftSkill>gewohnt, selbstständig und verantwortungsbewusst zu arbeiten</SoftSkill>.","[gewohnt, selbstständig, verantwortungsbewusst zu arbeiten]","[gewohnt zu arbeiten, selbstständig zu arbeiten, verantwortungsbewusst zu arbeiten]","[gewohnt, selbstständig und verantwortungsbewusst zu arbeiten]",3,1,0.0,0.0,"[[{'rouge-1': {'r': 0.3333333333333333, 'p': 0.6666666666666666, 'f': 0.44444444000000005}, 'rouge-2': {'r': 0.2, 'p': 0.5, 'f': 0.2857142816326531}, 'rouge-l': {'r': 0.3333333333333333, 'p': 0.6666666666666666, 'f': 0.44444444000000005}}]]",[],0.444444,0.285714,0.444444,-1.0,0,True,MC
288,832,"es sich <SoftSkill_C>gewohnt</SoftSkill_C>, <SoftSkill_C>sorgfältig</SoftSkill_C> und <SoftSkill>sehr selbstständig zu arbeiten</SoftSkill>? ->","es sich <SoftSkill>gewohnt</SoftSkill>, <SoftSkill>sorgfältig zu arbeiten </SoftSkill>und <SoftSkill>sehr selbstständig zu arbeiten</SoftSkill>?","es sich <SoftSkill>gewohnt, sorgfältig und sehr se

In [270]:
pd.set_option('display.max_row', None)
results_C['levenshtein_skill_C'].mean()

1.1027397260273972

## Sentence ROUGE

In [271]:
from rouge import Rouge

# Function to calculate Rouge scores
def calculate_rouge_scores(reference, candidate):
    rouge = Rouge()
    scores = rouge.get_scores(reference, candidate)
    rouge_1_f = scores[0]['rouge-1']['f'] if 'rouge-1' in scores[0] else None
    rouge_2_f = scores[0]['rouge-2']['f'] if 'rouge-2' in scores[0] else None
    rouge_l_f = scores[0]['rouge-l']['f'] if 'rouge-l' in scores[0] else None
    return rouge_1_f, rouge_2_f, rouge_l_f

# Apply the functions to calculate the scores and add them as new columns
results_SENTENCE['rouge1_score'], results_SENTENCE['rouge2_score'], results_SENTENCE['rougeL_score'] = zip(*results_SENTENCE.apply(
    lambda row: calculate_rouge_scores(row['cleaned_expected_text'], row['cleaned_generated_text']),
    axis=1
))

/tmp/ipykernel_3970478/809110196.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_SENTENCE['rouge1_score'], results_SENTENCE['rouge2_score'], results_SENTENCE['rougeL_score'] = zip(*results_SENTENCE.apply(
/tmp/ipykernel_3970478/809110196.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_SENTENCE['rouge1_score'], results_SENTENCE['rouge2_score'], results_SENTENCE['rougeL_score'] = zip(*results_SENTENCE.apply(
/tmp/ipykernel_3970478/809110196.py:13: SettingWithCopyWarning: 
A value is 

## Sentence BERT Cosine Similarity

In [272]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch

# Assuming you have already read the DataFrame from the CSV file
# results_SENTENCE = pd.read_csv("/path/to/results_SENTENCE.csv")

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to calculate cosine similarity
def calculate_cosine_similarity(sentence1, sentence2):
    # Tokenize and convert sentences to tensors
    inputs = tokenizer([sentence1, sentence2], return_tensors="pt", padding=True, truncation=True)

    # Get BERT embeddings
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :].numpy()

    # Normalize embeddings
    normalized_embeddings = embeddings / (embeddings ** 2).sum(axis=-1, keepdims=True) ** 0.5

    # Calculate cosine similarity
    similarity = (normalized_embeddings[0] @ normalized_embeddings[1].T).item()
    return similarity

# Apply the function to calculate cosine similarity and add it as a new column
results_SENTENCE['cosine_similarity'] = results_SENTENCE.apply(lambda row: calculate_cosine_similarity(row['cleaned_generated_text'], row['cleaned_expected_text']), axis=1)


/tmp/ipykernel_3970478/2919956664.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_SENTENCE['cosine_similarity'] = results_SENTENCE.apply(lambda row: calculate_cosine_similarity(row['cleaned_generated_text'], row['cleaned_expected_text']), axis=1)


## FUnction for cosine similarity for _C and ALL

In [273]:

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to get BERT embeddings for a list of sentences
def get_bert_embeddings(sentences):
    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :].numpy()
    return embeddings

# Function to calculate cosine similarity between two lists of sentences
def calculate_cosine_similarity(list1, list2):
    if not list1 or not list2:
        return np.nan

    # Get BERT embeddings for both lists of sentences
    embeddings1 = get_bert_embeddings(list1)
    embeddings2 = get_bert_embeddings(list2)

    # Normalize embeddings
    normalized_embeddings1 = embeddings1 / (embeddings1 ** 2).sum(axis=-1, keepdims=True) ** 0.5
    normalized_embeddings2 = embeddings2 / (embeddings2 ** 2).sum(axis=-1, keepdims=True) ** 0.5

    # Calculate cosine similarity for each sentence pair
    similarity_values = np.dot(normalized_embeddings1, normalized_embeddings2.T)
    return similarity_values.diagonal().tolist()


In [274]:

# Apply the function to calculate pairwise cosine similarity and store the distances in a list
cosine_similarity_list = []
for index, row in results_C.iterrows():
    list1 = row['expected_skills_C']
    list2 = row['generated_skills_C']

    # Calculate cosine similarity for each sentence pair in both lists
    pairwise_similarities = calculate_cosine_similarity(list1, list2)

    # Append the list of pairwise similarities for the row
    cosine_similarity_list.append(pairwise_similarities)

# Add the list of pairwise cosine similarity distances as a new column in the DataFrame
results_C['pairwise_cosine_similarity'] = cosine_similarity_list



# Apply the function to calculate pairwise cosine similarity and store the distances in a list
cosine_similarity_list = []
for index, row in results_ALL.iterrows():
    list1 = row['expected_skills_all']
    list2 = row['generated_skills_all']

    # Calculate cosine similarity for each sentence pair in both lists
    pairwise_similarities = calculate_cosine_similarity(list1, list2)

    # Append the list of pairwise similarities for the row
    cosine_similarity_list.append(pairwise_similarities)

# Add the list of pairwise cosine similarity distances as a new column in the DataFrame
results_ALL['pairwise_cosine_similarity'] = cosine_similarity_list

results_C['average_cosine_similarity'] = results_C['pairwise_cosine_similarity'].apply(lambda x: np.mean(x) if x else np.nan)
results_ALL['average_cosine_similarity'] = results_ALL['pairwise_cosine_similarity'].apply(lambda x: np.mean(x) if x else np.nan)


/tmp/ipykernel_3970478/2476177218.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_C['pairwise_cosine_similarity'] = cosine_similarity_list
/tmp/ipykernel_3970478/2476177218.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_ALL['pairwise_cosine_similarity'] = cosine_similarity_list
/tmp/ipykernel_3970478/2476177218.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [275]:
# results_ALL[results_ALL['mismatched_ALL']==True].head(5)

## Prepare Metrics and Tables 

In [276]:
# Create a DataFrame for _C metrics
summary_C_df = pd.DataFrame({
    'Metric': ['Average Rouge-1 Score (C)', 'Average Rouge-2 Score (C)',
               'Average Rouge-L Score (C)',  'Average Levenshtein (C)',
               'Weighted Avg. Percentage Skills (C)', 'Cosine Similarity(C)'],
    'Value': [results_C['rouge1_avg_skill_C'].mean(), results_C['rouge2_avg_skill_C'].mean(),
              results_C['rougeL_avg_skill_C'].mean(),results_C['levenshtein_skill_C'].mean(),
              results_C['num_skills_match_C'].sum() / results_C['num_skills_C'].sum(),results_C['average_cosine_similarity'].mean()]
})

# Create a DataFrame for "ALL" metrics
summary_ALL_df = pd.DataFrame({
    'Metric': ['Average Rouge-1 Score (ALL)', 'Average Rouge-2 Score (ALL)',
               'Average Rouge-L Score (ALL)', 'Average Levenshtein (ALL)',
               'Weighted Avg. Percentage Skills (ALL)', 'Cosine Similarity(ALL)'],
    'Value': [results_ALL['rouge1_avg_skill_all'].mean(), results_ALL['rouge2_avg_skill_all'].mean(),
              results_ALL['rougeL_avg_skill_all'].mean(), results_ALL['levenshtein_skill_all'].mean(),
              results_ALL['num_skills_match_all'].sum() / results_ALL['num_skills_all'].sum(),results_ALL['average_cosine_similarity'].mean()]
})

# Create a DataFrame for "ALL" metrics
summary_SENTENCE_df = pd.DataFrame({
    'Metric': ['Average Rouge-1 Score (SENTENCE)', 'Average Rouge-2 Score (SENTENCE)',
               'Average Rouge-L Score (SENTENCE)', 'Average Levenshtein (SENTENCE)',
               'Cosine Similarity(SENTENCE)'],
    'Value': [results_SENTENCE['rouge1_score'].mean(), results_SENTENCE['rouge2_score'].mean(),
              results_SENTENCE['rougeL_score'].mean(), results_SENTENCE['sentence_levenshtein'].mean(),
              results_SENTENCE['cosine_similarity'].mean()]
})

# Export the summary DataFrames to CSV
# summary_C_df.to_csv("summary_metrics_C.csv", index=False)
# summary_ALL_df.to_csv("summary_metrics_ALL.csv", index=False)

In [277]:
summary_C_df

,Metric,Value
0,Average Rouge-1 Score (C),0.935955
1,Average Rouge-2 Score (C),0.874133
2,Average Rouge-L Score (C),0.935955
3,Average Levenshtein (C),1.102740
4,Weighted Avg. Percentage Skills (C),0.925187
5,Cosine Similarity(C),0.996668


In [278]:
summary_ALL_df

,Metric,Value
0,Average Rouge-1 Score (ALL),0.978463
1,Average Rouge-2 Score (ALL),0.879541
2,Average Rouge-L Score (ALL),0.978463
3,Average Levenshtein (ALL),1.417808
4,Weighted Avg. Percentage Skills (ALL),0.966351
5,Cosine Similarity(ALL),0.997079


In [279]:
summary_SENTENCE_df

,Metric,Value
0,Average Rouge-1 Score (SENTENCE),0.995703
1,Average Rouge-2 Score (SENTENCE),0.985159
2,Average Rouge-L Score (SENTENCE),0.995703
3,Average Levenshtein (SENTENCE),1.756849
4,Cosine Similarity(SENTENCE),0.998346


## Problem Types DATA RESULTS _C SKILLS

In [280]:
summary_C_df_list = []

# Group the DataFrame by "Problem Type"
grouped_results_C = results_C.groupby('Problem type')

# Iterate over each group and calculate the summary DataFrame
for problem_type, group in grouped_results_C:
    num_samples = len(group)  # Calculate the number of samples for the current category
    summary_C_df = pd.DataFrame({
        'Average Rouge-1 Score (C)': [group['rouge1_avg_skill_C'].mean()],
        'Average Rouge-2 Score (C)': [group['rouge2_avg_skill_C'].mean()],
        'Average Rouge-L Score (C)': [group['rougeL_avg_skill_C'].mean()],
        'Average Levenshtein (C)': [group['levenshtein_skill_C'].mean()],
        'Weighted Avg. Percentage Skills (C)': [group['num_skills_match_C'].sum() / group['num_skills_C'].sum()],
        'Cosine Similarity (C)': [group['average_cosine_similarity'].mean()],
        'Number of Samples': [num_samples]
    }, index=[problem_type])
    # Add the summary DataFrame for the current category to the list
    summary_C_df_list.append(summary_C_df)

# Concatenate the summary DataFrames into a single DataFrame
result_summary_df = pd.concat(summary_C_df_list)

# Calculate overall averages across all categories and add as a new row
overall_averages = {
    'Average Rouge-1 Score (C)': results_C['rouge1_avg_skill_C'].mean(),
    'Average Rouge-2 Score (C)': results_C['rouge2_avg_skill_C'].mean(),
    'Average Rouge-L Score (C)': results_C['rougeL_avg_skill_C'].mean(),
    'Average Levenshtein (C)': results_C['levenshtein_skill_C'].mean(),
    'Weighted Avg. Percentage Skills (C)': results_C['num_skills_match_C'].sum() / results_C['num_skills_C'].sum(),
    'Cosine Similarity (C)': results_C['average_cosine_similarity'].mean(),
    'Number of Samples': len(results_C)
}

overall_summary_df = pd.DataFrame(overall_averages, index=['All Samples'])

# Concatenate the overall summary as a new row to the result_summary_df
result_summary_df_C_skills = pd.concat([result_summary_df, overall_summary_df])

result_summary_df_C_skills['model'] = 'GPT3-part-3'
result_summary_df_C_skills


,Average Rouge-1 Score (C),Average Rouge-2 Score (C),Average Rouge-L Score (C),Average Levenshtein (C),Weighted Avg. Percentage Skills (C),Cosine Similarity (C),Number of Samples,model
HMC,0.750000,0.604167,0.750000,0.000000,0.888889,1.000000,8,GPT3-part-3
HMDC,1.000000,0.800000,1.000000,0.000000,1.000000,1.000000,5,GPT3-part-3
HOWA,1.000000,0.666667,1.000000,0.000000,1.000000,1.000000,30,GPT3-part-3
MC,0.958000,0.950000,0.958000,0.240000,0.971429,0.999527,50,GPT3-part-3
MDC,0.809486,0.751010,0.809486,3.433333,0.787879,0.988108,30,GPT3-part-3
MOWA,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1,GPT3-part-3
MWA,0.947619,0.923990,0.947619,2.287879,0.909091,0.993180,66,GPT3-part-3
NONE,0.857143,0.466667,0.857143,7.666667,0.666667,0.985404,3,GPT3-part-3
OWA,0.948980,0.938776,0.948980,0.336735,0.959596,0.998929,98,GPT3-part-3
All Samples,0.935955,0.874133,0.935955,1.102740,0.925187,0.996668,292,GPT3-part-3


In [289]:
result_summary_df_C_skills['Problem Type'] = result_summary_df_C_skills.index
new_df = result_summary_df_C_skills.iloc[:, [2, 3, 4, 5, 8]]
latex_table = new_df.to_latex(index=False)
print(latex_table)

\begin{tabular}{rrrrl}
\toprule
Average Rouge-L Score (C) & Average Levenshtein (C) & Weighted Avg. Percentage Skills (C) & Cosine Similarity (C) & Problem Type \\
\midrule
0.750000 & 0.000000 & 0.888889 & 1.000000 & HMC \\
1.000000 & 0.000000 & 1.000000 & 1.000000 & HMDC \\
1.000000 & 0.000000 & 1.000000 & 1.000000 & HOWA \\
0.958000 & 0.240000 & 0.971429 & 0.999527 & MC \\
0.809486 & 3.433333 & 0.787879 & 0.988108 & MDC \\
1.000000 & 0.000000 & 1.000000 & 1.000000 & MOWA \\
0.947619 & 2.287879 & 0.909091 & 0.993180 & MWA \\
0.857143 & 7.666667 & 0.666667 & 0.985404 & NONE \\
0.948980 & 0.336735 & 0.959596 & 0.998929 & OWA \\
0.935955 & 1.102740 & 0.925187 & 0.996668 & All Samples \\
\bottomrule
\end{tabular}



## Problem Types DATA RESULTS ALL SKILLS

In [282]:
# Create an empty list to store the summary DataFrames for each category
summary_ALL_df_list = []

# Group the DataFrame by "Problem Type"
grouped_results_ALL = results_ALL.groupby('Problem type')

# Iterate over each group and calculate the summary DataFrame
for problem_type, group in grouped_results_ALL:
    num_samples = len(group)  # Calculate the number of samples for the current category
    summary_ALL_df = pd.DataFrame({
        'Average Rouge-1 Score (ALL)': [group['rouge1_avg_skill_all'].mean()],
        'Average Rouge-2 Score (ALL)': [group['rouge2_avg_skill_all'].mean()],
        'Average Rouge-L Score (ALL)': [group['rougeL_avg_skill_all'].mean()],
        'Average Levenshtein (ALL)': [group['levenshtein_skill_all'].mean()],
        'Weighted Avg. Percentage Skills (ALL)': [group['num_skills_match_all'].sum() / group['num_skills_all'].sum()],
        'Cosine Similarity (ALL)': [group['average_cosine_similarity'].mean()],
        'Number of Samples': [num_samples]
    }, index=[problem_type])
    # Add the summary DataFrame for the current category to the list
    summary_ALL_df_list.append(summary_ALL_df)

# Concatenate the summary DataFrames into a single DataFrame
result_summary_ALL_df = pd.concat(summary_ALL_df_list)

# Calculate overall averages across all categories and append as a new row
overall_averages_ALL = {
    'Average Rouge-1 Score (ALL)': results_ALL['rouge1_avg_skill_all'].mean(),
    'Average Rouge-2 Score (ALL)': results_ALL['rouge2_avg_skill_all'].mean(),
    'Average Rouge-L Score (ALL)': results_ALL['rougeL_avg_skill_all'].mean(),
    'Average Levenshtein (ALL)': results_ALL['levenshtein_skill_all'].mean(),
    'Weighted Avg. Percentage Skills (ALL)': results_ALL['num_skills_match_all'].sum() / results_ALL['num_skills_all'].sum(),
    'Cosine Similarity (ALL)': results_ALL['average_cosine_similarity'].mean(),
    'Number of Samples': len(results_ALL)
}

overall_summary_ALL_df = pd.DataFrame(overall_averages_ALL, index=['All Samples'])

# Concatenate the overall summary as a new row to the result_summary_ALL_df
result_summary_df_ALL_skills = pd.concat([result_summary_ALL_df, overall_summary_ALL_df])

result_summary_df_ALL_skills['model'] = 'GPT3-part-3'
result_summary_df_ALL_skills


,Average Rouge-1 Score (ALL),Average Rouge-2 Score (ALL),Average Rouge-L Score (ALL),Average Levenshtein (ALL),Weighted Avg. Percentage Skills (ALL),Cosine Similarity (ALL),Number of Samples,model
HMC,0.915625,0.666667,0.915625,0.000000,0.939394,0.992822,8,GPT3-part-3
HMDC,1.000000,0.786667,1.000000,0.000000,1.000000,1.000000,5,GPT3-part-3
HOWA,0.997222,0.575000,0.997222,0.166667,0.989247,0.999266,30,GPT3-part-3
MC,0.987148,0.957095,0.987148,0.320000,0.975000,0.998594,50,GPT3-part-3
MDC,0.935188,0.860213,0.935188,3.833333,0.917910,0.990056,30,GPT3-part-3
MOWA,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1,GPT3-part-3
MWA,0.969012,0.907125,0.969012,2.984848,0.960452,0.995768,66,GPT3-part-3
NONE,0.928571,0.733333,0.928571,7.666667,0.833333,0.992702,3,GPT3-part-3
OWA,0.993022,0.944671,0.993022,0.591837,0.982332,0.998942,98,GPT3-part-3
All Samples,0.978463,0.879541,0.978463,1.417808,0.966351,0.997079,292,GPT3-part-3


In [288]:
result_summary_df_ALL_skills['Problem Type'] = result_summary_df_ALL_skills.index
new_df = result_summary_df_ALL_skills.iloc[:, [2, 3, 4, 5, 8]]
latex_table = new_df.to_latex(index=False)
print(latex_table)

\begin{tabular}{rrrrl}
\toprule
Average Rouge-L Score (ALL) & Average Levenshtein (ALL) & Weighted Avg. Percentage Skills (ALL) & Cosine Similarity (ALL) & Problem Type \\
\midrule
0.915625 & 0.000000 & 0.939394 & 0.992822 & HMC \\
1.000000 & 0.000000 & 1.000000 & 1.000000 & HMDC \\
0.997222 & 0.166667 & 0.989247 & 0.999266 & HOWA \\
0.987148 & 0.320000 & 0.975000 & 0.998594 & MC \\
0.935188 & 3.833333 & 0.917910 & 0.990056 & MDC \\
1.000000 & 0.000000 & 1.000000 & 1.000000 & MOWA \\
0.969012 & 2.984848 & 0.960452 & 0.995768 & MWA \\
0.928571 & 7.666667 & 0.833333 & 0.992702 & NONE \\
0.993022 & 0.591837 & 0.982332 & 0.998942 & OWA \\
0.978463 & 1.417808 & 0.966351 & 0.997079 & All Samples \\
\bottomrule
\end{tabular}



## Problem Types DATA RESULTS SENTENCE SKILLS

In [284]:
# Create an empty list to store the summary DataFrames for each category
summary_SENTENCE_df_list = []

# Group the DataFrame by "Problem Type"
grouped_results_SENTENCE = results_SENTENCE.groupby('Problem type')

# Iterate over each group and calculate the summary DataFrame
for problem_type, group in grouped_results_SENTENCE:
    num_samples = len(group)  # Calculate the number of samples for the current category
    summary_SENTENCE_df = pd.DataFrame({
        'Average Rouge-1 Score (SENTENCE)': [group['rouge1_score'].mean()],
        'Average Rouge-2 Score (SENTENCE)': [group['rouge2_score'].mean()],
        'Average Rouge-L Score (SENTENCE)': [group['rougeL_score'].mean()],
        'Average Levenshtein (SENTENCE)': [group['sentence_levenshtein'].mean()],
        'Cosine Similarity (SENTENCE)': [group['cosine_similarity'].mean()],
        'Number of Samples': [num_samples]
    }, index=[problem_type])
    # Add the summary DataFrame for the current category to the list
    summary_SENTENCE_df_list.append(summary_SENTENCE_df)

# Concatenate the summary DataFrames into a single DataFrame
result_summary_SENTENCE_df = pd.concat(summary_SENTENCE_df_list)

# Calculate overall averages across all categories and add as a new row
overall_averages_SENTENCE = {
    'Average Rouge-1 Score (SENTENCE)': results_SENTENCE['rouge1_score'].mean(),
    'Average Rouge-2 Score (SENTENCE)': results_SENTENCE['rouge2_score'].mean(),
    'Average Rouge-L Score (SENTENCE)': results_SENTENCE['rougeL_score'].mean(),
    'Average Levenshtein (SENTENCE)': results_SENTENCE['sentence_levenshtein'].mean(),
    'Cosine Similarity (SENTENCE)': results_SENTENCE['cosine_similarity'].mean(),
    'Number of Samples': len(results_SENTENCE)
}

overall_summary_SENTENCE_df = pd.DataFrame(overall_averages_SENTENCE, index=['All Samples'])

# Concatenate the overall summary as a new row to the result_summary_SENTENCE_df
result_summary_df_SENTENCE = pd.concat([result_summary_SENTENCE_df, overall_summary_SENTENCE_df])

result_summary_df_SENTENCE['model'] = 'GPT3-part-3'
result_summary_df_SENTENCE

,Average Rouge-1 Score (SENTENCE),Average Rouge-2 Score (SENTENCE),Average Rouge-L Score (SENTENCE),Average Levenshtein (SENTENCE),Cosine Similarity (SENTENCE),Number of Samples,model
HMC,1.000000,0.992647,1.000000,1.875000,0.999825,8,GPT3-part-3
HMDC,1.000000,1.000000,1.000000,0.000000,1.000000,5,GPT3-part-3
HOWA,1.000000,0.994737,1.000000,0.166667,0.999827,30,GPT3-part-3
MC,0.997143,0.987956,0.997143,0.840000,0.999462,50,GPT3-part-3
MDC,0.995215,0.965999,0.995215,4.100000,0.998578,30,GPT3-part-3
MOWA,1.000000,1.000000,1.000000,0.000000,1.000000,1,GPT3-part-3
MWA,0.990576,0.979959,0.990576,3.696970,0.996517,66,GPT3-part-3
NONE,1.000000,0.956522,1.000000,7.666667,0.991321,3,GPT3-part-3
OWA,0.996467,0.989373,0.996467,0.622449,0.998459,98,GPT3-part-3
All Samples,0.995703,0.985159,0.995703,1.756849,0.998346,292,GPT3-part-3


In [287]:
result_summary_df_SENTENCE['Problem Type'] = result_summary_df_SENTENCE.index
new_df = result_summary_df_SENTENCE.iloc[:, [2, 3, 4, 7]]
latex_table = new_df.to_latex(index=False)
print(latex_table)

\begin{tabular}{rrrl}
\toprule
Average Rouge-L Score (SENTENCE) & Average Levenshtein (SENTENCE) & Cosine Similarity (SENTENCE) & Problem Type \\
\midrule
1.000000 & 1.875000 & 0.999825 & HMC \\
1.000000 & 0.000000 & 1.000000 & HMDC \\
1.000000 & 0.166667 & 0.999827 & HOWA \\
0.997143 & 0.840000 & 0.999462 & MC \\
0.995215 & 4.100000 & 0.998578 & MDC \\
1.000000 & 0.000000 & 1.000000 & MOWA \\
0.990576 & 3.696970 & 0.996517 & MWA \\
1.000000 & 7.666667 & 0.991321 & NONE \\
0.996467 & 0.622449 & 0.998459 & OWA \\
0.995703 & 1.756849 & 0.998346 & All Samples \\
\bottomrule
\end{tabular}



In [286]:
# result_summary_df_C_skills.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/GPT3_SS/summary_metrics_C_GPT3-part-2.csv", index=False)
# result_summary_df_ALL_skills.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/GPT3_SS/summary_metrics_ALL_GPT3-part-2.csv", index=False)
# result_summary_df_SENTENCE.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/GPT3_SS/summary_metrics_SENTENCE_GPT3-part-2.csv", index=False)